In [0]:
import pandas as pd
import numpy as np
import math as mt
import sklearn as sk1
import datetime as dt
import matplotlib.pyplot as plt
import math 
import seaborn as plt1
from sklearn.model_selection import train_test_split
import pickle

!pip install missingpy
import missingpy as imput_helper


In [0]:
work_dat_notDead_large = pd.read_csv("work_dat_notDead_large-Final model specific.csv")


work_dat_notDead_large_backup = work_dat_notDead_large.copy()

work_dat_notDead_large = work_dat_notDead_large.loc[work_dat_notDead_large['InscClaimAmtReimbursed']>20000,:].copy()


In [0]:
# Applying models
from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import BayesianRidge
import lightgbm as lgbm
from sklearn.preprocessing import RobustScaler  # Here we are applying the RobustScaler to minimize variance and outliers in the data and make them more standard
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

#work_dat_notDead_large = work_dat_notDead_large.drop(columns=['BeneID'])

scaler = StandardScaler()
ClassFraud_y = work_dat_notDead_large['PotentialFraud'].copy()
ClassFraud_x = work_dat_notDead_large[work_dat_notDead_large.columns.difference(['PotentialFraud'])]

scaler.fit(ClassFraud_x)

# save the scaler
pickle.dump(scaler, open('fisher_scaler.pkl', 'wb'))

ClassFraud_x_Scaled = scaler.transform(ClassFraud_x)

X_train, X_val, y_train, y_val = train_test_split(ClassFraud_x_Scaled, ClassFraud_y, test_size=0.33)


In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
fisher = LinearDiscriminantAnalysis() 

param_grid = { 
    'solver' : ['svd','Isqr']
}

CV_rfc = GridSearchCV(estimator=fisher, param_grid=param_grid, cv= 3, scoring='roc_auc', n_jobs=-1)
CV_rfc.fit(X_train, y_train)
#print CV_rfc.best_params_


GridSearchCV(cv=3, error_score=nan,
             estimator=LinearDiscriminantAnalysis(n_components=None,
                                                  priors=None, shrinkage=None,
                                                  solver='svd',
                                                  store_covariance=False,
                                                  tol=0.0001),
             iid='deprecated', n_jobs=-1,
             param_grid={'solver': ['svd', 'Isqr']}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='roc_auc',
             verbose=0)

In [0]:
# save the model
pickle.dump(CV_rfc, open('fisher_model.pkl', 'wb'))

loaded_model_FD = pickle.load(open("fisher_model.pkl", 'rb'))

y_pred = loaded_model_FD.predict(X_val)

In [0]:
# Fisher discriminant Analysis
from sklearn.metrics import f1_score

f1_score(y_val,y_pred)

0.8552873538155149